In [1]:
using MLJ
using CSV, DataFrames
using Plots

The `@pipeline` macro is great for composing a linear sequence of models, but for more complicated composition we will want MLJ's generic model composition syntax. There are 2 main stes: 
- **Prototype** the composite model by building a *learning network* that can be tested on dummy data as it's built
- **Export** the learning network as a stand-alone model type

# Building a pipeline with the generic composition syntax

we will do the equivalent of 
```julia
pipe = @pipeline Standardizer LogisticClassifier;
```

In [2]:
# dummy data
X, y = make_blobs(5, 3) 
pretty(X)

┌────────────┬────────────┬────────────┐
│ x1         │ x2         │ x3         │
│ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┤
│ 9.19885    │ 4.36257    │ -5.20153   │
│ 8.30787    │ 4.93404    │ -5.7847    │
│ 6.5611     │ 5.36142    │ -3.13666   │
│ -9.25297   │ -15.7003   │ -1.90303   │
│ 0.573373   │ -4.72411   │ -7.81783   │
└────────────┴────────────┴────────────┘


# Step 0: combine models "by hand" 

In [3]:
LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
PCA = @load PCA

import MLJLinearModels ✔
import MLJMultivariateStatsInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/john/.julia/packages/MLJModels/GKDnU/src/loading.jl:168
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/john/.julia/packages/MLJModels/GKDnU/src/loading.jl:168


MLJMultivariateStatsInterface.PCA

In [4]:
stand = Standardizer()
linear = LogisticClassifier()

LogisticClassifier(
    lambda = 1.0,
    gamma = 0.0,
    penalty = :l2,
    fit_intercept = true,
    penalize_intercept = false,
    solver = nothing)

In [6]:
mach1 = machine(stand, X); 
fit!(mach1); 
Xstand = MLJ.transform(mach1, X);

┌ Info: Training Machine{Standardizer,…}.
└ @ MLJBase /home/john/.julia/packages/MLJBase/QXObv/src/machines.jl:403


In [7]:
mach2 = machine(linear, Xstand, y); 
fit!(mach2); 
yhat = predict(mach2, Xstand)

┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase /home/john/.julia/packages/MLJBase/QXObv/src/machines.jl:403


5-element MLJBase.UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.0483, 2=>0.097, 3=>0.855)
 UnivariateFinite{Multiclass{3}}(1=>0.0474, 2=>0.124, 3=>0.828)
 UnivariateFinite{Multiclass{3}}(1=>0.0714, 2=>0.0499, 3=>0.879)
 UnivariateFinite{Multiclass{3}}(1=>0.78, 2=>0.106, 3=>0.113)
 UnivariateFinite{Multiclass{3}}(1=>0.128, 2=>0.547, 3=>0.325)

# Step 1: Edit your code
- pre-wrap the data in `Source` nodes 
- delete the `fit!` calls

In [8]:
X = source(X) 
y = source(y) 

stand = Standardizer(); 
linear = LogisticClassifier(); 

mach1 = machine(stand, X); 
Xstand = MLJ.transform(mach1, X); 

mach2 = machine(linear, Xstand, y); 
yhat = predict(mach2, Xstand) 

Node{Machine{LogisticClassifier,…}}
  args:
    1:	Node{Machine{Standardizer,…}}
  formula:
    predict(
        Machine{LogisticClassifier,…}, 
        transform(
            Machine{Standardizer,…}, 
            Source @053))

`X`, `y`, `Xstand`, and `yhat` are *nodes* (i.e. variables or *dynamic data*). All training, predicting, and transforming is now executed *lazily* on demand when we call `fit!`. We call a node to retrieve the data it represents. 

In [9]:
fit!(Xstand) 
Xstand() |> pretty

┌────────────┬────────────┬────────────┐
│ x1         │ x2         │ x3         │
│ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┤
│ 0.798141   │ 0.603367   │ -0.18714   │
│ 0.681967   │ 0.665879   │ -0.439311  │
│ 0.454206   │ 0.71263    │ 0.705741   │
│ -1.60778   │ -1.59127   │ 1.23918    │
│ -0.326531  │ -0.390607  │ -1.31847   │
└────────────┴────────────┴────────────┘


┌ Info: Training Machine{Standardizer,…}.
└ @ MLJBase /home/john/.julia/packages/MLJBase/QXObv/src/machines.jl:403


In [10]:
fit!(yhat); 
yhat()

┌ Info: Not retraining Machine{Standardizer,…}. Use `force=true` to force.
└ @ MLJBase /home/john/.julia/packages/MLJBase/QXObv/src/machines.jl:406
┌ Info: Training Machine{LogisticClassifier,…}.
└ @ MLJBase /home/john/.julia/packages/MLJBase/QXObv/src/machines.jl:403


5-element MLJBase.UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.0483, 2=>0.097, 3=>0.855)
 UnivariateFinite{Multiclass{3}}(1=>0.0474, 2=>0.124, 3=>0.828)
 UnivariateFinite{Multiclass{3}}(1=>0.0714, 2=>0.0499, 3=>0.879)
 UnivariateFinite{Multiclass{3}}(1=>0.78, 2=>0.106, 3=>0.113)
 UnivariateFinite{Multiclass{3}}(1=>0.128, 2=>0.547, 3=>0.325)

since the network is a DAG (directed acyclic graph), we can inspect the predecessors of each node: 

In [11]:
sources(yhat)

2-element Vector{Any}:
 Source @053 ⏎ `Table{AbstractVector{Continuous}}`
 Source @100 ⏎ `AbstractVector{Multiclass{3}}`

To obtain a new prediction, we can call `yhat(Xnew)`. 

In [13]:
Xnew, _ = make_blobs(2, 3); 
Xnew |> pretty

yhat(Xnew)

┌────────────┬────────────┬────────────┐
│ x1         │ x2         │ x3         │
│ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┤
│ -8.49929   │ -22.3569   │ -6.07796   │
│ -1.69238   │ -8.67298   │ -10.3831   │
└────────────┴────────────┴────────────┘


2-element MLJBase.UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.504, 2=>0.445, 3=>0.0507)
 UnivariateFinite{Multiclass{3}}(1=>0.0677, 2=>0.834, 3=>0.0978)

# Step 2: Export the learning network as a stand alone model

We can have 3 different types of models: 
- `Deterministic`
- `Probabilistic`
- `Unsupervised` 

We also supply the source nodes and prediction node

In [14]:
mach = machine(Probabilistic(), X, y; predict=yhat)

Machine{ProbabilisticSurrogate,…} trained 0 times; does not cache data
  args: 
    1:	Source @053 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @100 ⏎ `AbstractVector{Multiclass{3}}`


we can use this like a normal machine... 

In [15]:
Xnew, _ = make_blobs(2, 3); 
fit!(mach) 
predict(mach, Xnew)

┌ Info: Not retraining Machine{Standardizer,…}. Use `force=true` to force.
└ @ MLJBase /home/john/.julia/packages/MLJBase/QXObv/src/machines.jl:406
┌ Info: Not retraining Machine{LogisticClassifier,…}. Use `force=true` to force.
└ @ MLJBase /home/john/.julia/packages/MLJBase/QXObv/src/machines.jl:406


2-element MLJBase.UnivariateFiniteVector{Multiclass{3}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{3}}(1=>0.47, 2=>0.00127, 3=>0.529)
 UnivariateFinite{Multiclass{3}}(1=>0.441, 2=>0.00234, 3=>0.556)

now we create a new model type, a Julia `struct` 

In [16]:
@from_network mach begin
    mutable struct YourPipe
        standardizer = stand
        classifier = linear::Probabilistic
    end
end

now we can instantiate the new model on new data

In [17]:
pipe = YourPipe()
X, y = @load_iris; 

mach = machine(pipe, X, y) 
evaluate!(mach, measure=misclassification_rate, operation=predict_mode)

Evaluating over 6 folds: 100%[=========================] Time: 0:00:10


PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌─────────────────────────┬─────────────┬──────────────┬────────────────────────
│ measure                 │ measurement │ operation    │ per_fold              ⋯
├─────────────────────────┼─────────────┼──────────────┼────────────────────────
│ MisclassificationRate() │ 0.08        │ predict_mode │ [0.0, 0.04, 0.08, 0.0 ⋯
└─────────────────────────┴─────────────┴──────────────┴────────────────────────
                                                                1 column omitted


# A composite model to average two regressor predictors

We will define a model that 
- standardizes the input data
- learns and applies a Box-Cox transformation to target variable that enforces a normal distribution 
- blends the predictions of two supervised learning models using a simple average 
- applies the inverse Box-Cox transformation to the blended prediction

In [19]:
using Pkg 
Pkg.add("MLJDecisionTreeInterface")
Pkg.add("DecisionTree")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/gitRepos/ml-demos/Project.toml`
  [c6f25543] + MLJDecisionTreeInterface v0.1.3
    Updating `~/gitRepos/ml-demos/Manifest.toml`
  [7806a523] + DecisionTree v0.10.11
  [c6f25543] + MLJDecisionTreeInterface v0.1.3
  [6e75b9c4] + ScikitLearnBase v0.5.0
   Resolving package versions...
    Updating `~/gitRepos/ml-demos/Project.toml`
  [7806a523] + DecisionTree v0.10.11
  No Changes to `~/gitRepos/ml-demos/Manifest.toml`


In [24]:
RandomForestRegressor = @load RandomForestRegressor pkg=DecisionTree
RidgeRegressor = @load RidgeRegressor pkg=MLJLinearModels

import MLJDecisionTreeInterface ✔
import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/john/.julia/packages/MLJModels/GKDnU/src/loading.jl:168
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/john/.julia/packages/MLJModels/GKDnU/src/loading.jl:168


MLJLinearModels.RidgeRegressor

## Input Layer

In [21]:
X = source() 
y = source() 

Source @699 ⏎ `Nothing`

## First layer and target transformation

In [22]:
std_model = Standardizer() 
stand = machine(std_model, X) 
W = MLJ.transform(stand, X) 

Node{Machine{Standardizer,…}}
  args:
    1:	Source @413
  formula:
    transform(
        Machine{Standardizer,…}, 
        Source @413)

In [23]:
box_model = UnivariateBoxCoxTransformer()
box = machine(box_model, y) 
z = MLJ.transform(box, y) 

Node{Machine{UnivariateBoxCoxTransformer,…}}
  args:
    1:	Source @699
  formula:
    transform(
        Machine{UnivariateBoxCoxTransformer,…}, 
        Source @699)

## Second layer

In [25]:
ridge_model = RidgeRegressor(lambda=0.1)
ridge = machine(ridge_model, W, z) 

forest_model = RandomForestRegressor(n_trees=50)
forest = machine(forest_model, W, z) 

Machine{RandomForestRegressor,…} trained 0 times; caches data
  args: 
    1:	Node{Machine{Standardizer,…}}
    2:	Node{Machine{UnivariateBoxCoxTransformer,…}}


In [26]:
ẑ = 0.5*predict(ridge, W) + 0.5*predict(forest, W)

Node{Nothing}
  args:
    1:	Node{Nothing}
    2:	Node{Nothing}
  formula:
    +(
        #146(
            predict(
                Machine{RidgeRegressor,…}, 
                transform(
                    Machine{Standardizer,…}, 
                    Source @413))),
        #146(
            predict(
                Machine{RandomForestRegressor,…}, 
                transform(
                    Machine{Standardizer,…}, 
                    Source @413))))

## Output

In [27]:
ŷ = inverse_transform(box, ẑ)

Node{Machine{UnivariateBoxCoxTransformer,…}}
  args:
    1:	Node{Nothing}
  formula:
    inverse_transform(
        Machine{UnivariateBoxCoxTransformer,…}, 
        +(
            #146(
                predict(
                    Machine{RidgeRegressor,…}, 
                    transform(
                        Machine{Standardizer,…}, 
                        Source @413))),
            #146(
                predict(
                    Machine{RandomForestRegressor,…}, 
                    transform(
                        Machine{Standardizer,…}, 
                        Source @413)))))

## Export the new model 

In [28]:
@from_network machine(Deterministic(), X, y, predict=ŷ) begin
    mutable struct CompositeModel 
        rgs1 = ridge_model 
        rgs2 = forest_model
    end
end

Instantiate the new model and try it out

In [29]:
composite = CompositeModel()

CompositeModel(
    rgs1 = RidgeRegressor(
            lambda = 0.1,
            fit_intercept = true,
            penalize_intercept = false,
            solver = nothing),
    rgs2 = RandomForestRegressor(
            max_depth = -1,
            min_samples_leaf = 1,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = -1,
            n_trees = 50,
            sampling_fraction = 0.7,
            pdf_smoothing = 0.0,
            rng = Random._GLOBAL_RNG()))

In [30]:
X, y = @load_boston;
mach = machine(composite, X, y); 

evaluate!(mach, resampling=CV(nfolds=6, shuffle=true), measures=[rms, mae])

Evaluating over 6 folds: 100%[=========================] Time: 0:00:16


PerformanceEvaluation object with these fields:
  measure, measurement, operation, per_fold,
  per_observation, fitted_params_per_fold,
  report_per_fold, train_test_pairs
Extract:
┌────────────────────────┬─────────────┬───────────┬────────────────────────────
│ measure                │ measurement │ operation │ per_fold                  ⋯
├────────────────────────┼─────────────┼───────────┼────────────────────────────
│ RootMeanSquaredError() │ 3.88        │ predict   │ [4.03, 3.29, 4.5, 2.71, 4 ⋯
│ MeanAbsoluteError()    │ 2.49        │ predict   │ [2.65, 2.35, 2.57, 2.1, 2 ⋯
└────────────────────────┴─────────────┴───────────┴────────────────────────────
                                                                1 column omitted
